In [1]:
import os
import datetime
from typing import Optional, Tuple
import json
import numpy as np
import torch
from gymnasium.spaces import Box, Discrete
import pandas as pd 

import random

from torch.distributions import Normal, Distribution
import pandas as pd

from tianshou.data import Collector, VectorReplayBuffer, PrioritizedVectorReplayBuffer
from tianshou.env import SubprocVectorEnv, DummyVectorEnv
#from tianshou.env.pettingzoo_env_parallel import PettingZooParallelEnv
#from tianshou.env.pettingzoo_env import PettingZooEnv
#from PettingZooParallelEnv import PettingZooParal

from tianshou.policy import PPOPolicy
from tianshou.trainer import OnpolicyTrainer

from tianshou.utils.net.common import ActorCritic, DataParallelNet, Net
from tianshou.utils.net.continuous import Actor, Critic

from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, DDPGPolicy
from tianshou.trainer import OffpolicyTrainer
from torch.utils.tensorboard import SummaryWriter
from DNN_B_ACE_ACTOR import DNN_B_ACE_ACTOR
from DNN_B_ACE_CRITIC import DNN_B_ACE_CRITIC
from Task_MHA_B_ACE import Task_MHA_B_ACE
from Task_DNN_B_ACE import Task_DNN_B_ACE
from Task_B_ACE_Env import B_ACE_TaskEnv

from CollectorMA import CollectorMA
from MAParalellPolicy import MAParalellPolicy

import numpy as np
from scipy.stats import bootstrap


####---------------------------#######
#Tianshou Adjustment
import wandb
# os.environ["WANDB_NOTEBOOK_NAME"] = "Tianshow_Training_GoDot.ipybn"
from tianshou.utils import WandbLogger
# from tianshou.utils.logger.base import LOG_DATA_TYPE
# def new_write(self, step_type: str, step: int, data: LOG_DATA_TYPE) -> None:
#      data[step_type] = step
#      wandb.log(data)   
# WandbLogger.write = new_write 
####---------------------------#######


model  =  "Task_MHA_B_ACE"#"SISL_Task_MultiHead" #"CNN_ATT_SISL" #"MultiHead_SISL" 
test_num  =  "_B_ACE03"
policyModel  =  "DQN"
name = model + test_num

train_env_num = 4
test_env_num  = 15

now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = name + str(now)
log_path = os.path.join('./', "Logs", "dqn_sisl", log_name)

#Duck Training Best
#load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE02240721-151049_1261_BestRew.pth'

#FSMG1 Training Best
load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241019-215530_1893_BestRew.pth'



save_policy_name = f'policy_{log_name}'
policy_path = model + policyModel


model_load_path = os.path.join(policy_path, load_policy_name)  
model_save_path = os.path.join(policy_path, save_policy_name)        
os.makedirs(os.path.join(policy_path), exist_ok=True)  
os.makedirs(os.path.join(log_path), exist_ok=True)

Policy_Config = {
    "same_policy" : True,
    "load_model" : False,
    "freeze_CNN" : False     
                }

B_ACE_Config = { 	
                    "EnvConfig" : 
                    {
                        "task": "b_ace_v1",
                        "env_path": "../../BVR_AirCombat/bin/B_ACE_v13.exe",
                        "port": 12500,
                        "renderize": 0,
                        "debug_view": 0,
                        "phy_fps": 20,
                        "speed_up": 50000,
                        "max_cycles": 36000,
                        "experiment_mode"  : 0,
                        "parallel_envs": 1,	
                        "seed": 1,	
                        "action_repeat": 20,	
                        "action_type": "Low_Level_Continuous",                        
                        "stop_mission" : 1,
                        
                        
                        "RewardsConfig" : {
                                    "mission_factor": 0.001,				
                                    "missile_fire_factor": -0.1,		
                                    "missile_no_fire_factor": -0.001,
                                    "missile_miss_factor": -0.5,
                                    "detect_loss_factor": -0.1,
                                    "keep_track_factor": 0.001,
                                    "hit_enemy_factor": 3.0,
                                    "hit_own_factor": -5.0,			
                                    "mission_accomplished_factor": 10.0,			
                                }
                    },

                    "AgentsConfig" : 
                    {
                        "blue_agents": { 
                            "num_agents" : 1,
                            "mission"    : "DCA",
                            "beh_config" : {
                                            "dShot" : [1.04, 0.50, 1.09],
                                            "lCrank": [1.06, 0.98, 0.98],
                                            "lBreak": [1.05, 1.17, 0.45]
                                        },
                            "base_behavior": "external",                  
                            "init_position": {"x": 0.0, "y": 25000.0,"z": 30.0},
                            "offset_pos": {	"x": 0.0, "y": 0.0, "z": 0.0},
                            "init_hdg": 0.0,                        
                            "target_position": {"x": 0.0,"y": 25000.0,"z": 30.0},
                            "rnd_offset_range":{"x": 10.0,"y": 10000.0,"z": 5.0},				
                            "rnd_shot_dist_var": 0.025,
                            "rnd_crank_var": 0.025,
                            "rnd_break_var": 0.025,
                            "wez_models" : "res://assets/wez/Default_Wez_params.json"
                        },	
                        "red_agents":
                        { 
                            "num_agents" : 1, 
                            "base_behavior": "duck",
                            "mission"    : "striker",
                            "beh_config" : {
                                              "dShot" : [1.04, 1.04, 1.04], #[1.04, 0.50, 1.09]
                                              "lCrank": [1.06, 1.06, 1.06], #1.06, 0.98, 0.98
                                              "lBreak": [1.05, 1.05, 1.05] #1.05, 1.17, 0.45
                                          },
                            # "beh_config" : {
                            #                "dShot" : [1.04, 0.50, 1.09],
                            #                 "lCrank": [1.06, 0.98, 0.98],
                            #                 "lBreak": [1.05, 1.17, 0.45]
                            #             },
                            "init_position": {"x": 0.0,"y": 25000.0,"z": -30.0},
                            "offset_pos": {"x": 0.0,"y": 0.0,"z": 0.0},
                            "init_hdg" : 180.0,                        
                            "target_position": {"x": 0.0,"y": 25000.0,"z": 30.0},
                            "rnd_offset_range":{"x": 10.0,"y": 10000.0,"z": 5.0},				
                            "rnd_shot_dist_var": 0.025,
                            "rnd_crank_var": 0.025,
                            "rnd_break_var": 0.025,
                            "wez_models" : "res://assets/wez/Default_Wez_params.json"
                        }
                    }	
            }
#max_cycles = B_ACE_Config["max_cycles"]
#n_agents = 1#B_ACE_Config["n_pursuers"]

dqn_params =    {
                "discount_factor": 0.99, 
                "estimation_step": 180, 
                "target_update_freq": 6000 * 3 ,#max_cycles * n_agents,
                "reward_normalization" : False,
                "clip_loss_grad" : False,
                "optminizer": "Adam",
                "lr": 0.00005, 
                "max_tasks" : 30
                }

PPO_params= {    
                'action_scaling': True,
                'discount_factor': 0.98,
                'max_grad_norm': 0.5,
                'eps_clip': 0.2,
                'vf_coef': 0.5,
                'ent_coef': 0.01,
                'gae_lambda': 0.95,
                'reward_normalization': False, 
                'dual_clip': None,
                'value_clip': False,   
                'deterministic_eval': True,
                'advantage_normalization': False,
                'recompute_advantage': False,
                'action_bound_method': "clip",
                'lr_scheduler': None,
            }


trainer_params = {"max_epoch": 500,
                  "step_per_epoch": 18000,#5 * (150 * n_agents),
                  "step_per_collect": 6000,# * (10 * n_agents),
                  
                  "batch_size" : 1024,
                  
                  "update_per_step": 1 / (100), #Off-Policy Only (run after close a Collect (run many times as necessary to meet the value))
                  
                  "repeat_per_collect": 32, #On-Policy Only
                  
                  "episode_per_test": 30,                  
                  "tn_eps_max": 0.20,
                  "ts_eps_max": 0.01,
                  "warmup_size" : 1,
                  "train_envs" : train_env_num,
                  "test_envs" : test_env_num
}
#agent_learn = PPOPolicy(**policy_params)


runConfig = dqn_params
runConfig.update(Policy_Config)
runConfig.update(B_ACE_Config)
runConfig.update(trainer_params) 
runConfig.update(dqn_params)


def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
    policy_load_path = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    
    env = _get_env()       
    agent_observation_space = env.observation_space("agent_0")
   
    #print(env.action_space)
    #action_shape = 50#env.action_space.shape
    
    #print("ActionSPACE: ", env.action_space)
    action_space = env.action_space
    device="cuda" if torch.cuda.is_available() else "cpu"  

    agents = []        
    
    if Policy_Config["same_policy"]:
        policies_number = 1
    else:
        policies_number = len(env.agents)

    for _ in range(policies_number):      
        
        #print(agent_observation_space)
        
        if policyModel == "DQN":

            if model == "Task_MHA_B_ACE":
                net = Task_MHA_B_ACE(
                    #obs_shape=agent_observation_space.shape,                                                  
                    num_tasks = dqn_params["max_tasks"],
                    num_features_per_task= 14,                    
                    nhead = 4,
                    device="cuda" if torch.cuda.is_available() else "cpu"
                    
                ).to(device) 
            
            if model == "Task_DNN_B_ACE":
                net = Task_DNN_B_ACE(
                    #obs_shape=agent_observation_space.shape,                                                  
                    num_tasks = dqn_params["max_tasks"],
                    num_features_per_task= 14,                    
                    nhead = 4,
                    device="cuda" if torch.cuda.is_available() else "cpu"
                    
                ).to(device) 
                
            optim = torch.optim.Adam(net.parameters(), lr=dqn_params["lr"], weight_decay=0.0, amsgrad= True)       
            
            agent_learn = DQNPolicy(
                model=net,
                optim=optim,
                action_space = Discrete(dqn_params["max_tasks"]),
                discount_factor= dqn_params["discount_factor"],
                estimation_step=dqn_params["estimation_step"],
                target_update_freq=dqn_params["target_update_freq"],
                reward_normalization = dqn_params["reward_normalization"],
                clip_loss_grad = dqn_params["clip_loss_grad"]
            )                   
        
        elif model == "PPO_DNN":
            
            actor = DNN_B_ACE_ACTOR(
                obs_shape=agent_observation_space.shape[0],                
                action_shape=4,                
                device="cuda" if torch.cuda.is_available() else "cpu"                
            ).to(device)

            critic = DNN_B_ACE_CRITIC(
                obs_shape=agent_observation_space.shape[0],                
                action_shape=4,                
                device="cuda" if torch.cuda.is_available() else "cpu"                
            ).to(device)
            
                                    
            actor_critic = ActorCritic(actor, critic)
        
            # orthogonal initialization
            # for m in actor_critic.modules():
            #     if isinstance(m, torch.nn.Linear):
            #         torch.nn.init.orthogonal_(m.weight)
            #         torch.nn.init.zeros_(m.bias)            
            
            # dist = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([1.0])) 
                # define policy
            def dist(mu, sigma) -> Distribution:
                return Normal(mu, sigma)        
                
            #optim_actor  = torch.optim.Adam(netActor.parameters(),  lr=dqn_params["lr"], weight_decay=0.0, amsgrad= True )
            #optim_critic = torch.optim.Adam(netCritic.parameters(), lr=dqn_params["lr"], weight_decay=0.0, amsgrad= True )
            optim = torch.optim.Adam(actor_critic.parameters(), lr=dqn_params["lr"])
                    
            agent_learn = PPOPolicy(
                actor=actor,
                critic=critic,
                optim=optim,
                dist_fn=dist,                
                action_scaling  =       PPO_params['action_scaling'],
                discount_factor =       PPO_params['discount_factor'],
                max_grad_norm   =       PPO_params['max_grad_norm'],
                eps_clip        =       PPO_params['eps_clip'],
                vf_coef         =       PPO_params['vf_coef'],
                ent_coef        =       PPO_params['ent_coef'],
                gae_lambda      =       PPO_params['gae_lambda'],
                reward_normalization=   PPO_params['reward_normalization'],
                action_space    =  action_space,
                deterministic_eval=     PPO_params['deterministic_eval'],
                advantage_normalization=PPO_params['advantage_normalization'],
                recompute_advantage=    PPO_params['recompute_advantage'],
                action_bound_method=    PPO_params['action_bound_method'],
                lr_scheduler=None
            )
            
        if Policy_Config["load_model"] is True:
            # Load the saved checkpoint             
            agent_learn.load_state_dict(torch.load(model_load_path))
            print(f'Loaded-> {model_load_path}')
                   
        
        agents.append(agent_learn)

    if Policy_Config["same_policy"]:
        agents = [agents[0] for _ in range(len(env.agents))]
    else:
        for _ in range(len(env.agents) - policies_number):
            agents.append(agents[0])
    
    policy = MultiAgentPolicyManager(policies = agents, env=env)  
    #policy = MAParalellPolicy(policies = agents, env=env, device="cuda" if torch.cuda.is_available() else "cpu" )  
        
    return policy, optim, env.agents

def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""   
    
    B_ACE_Config["EnvConfig"]["seed"] = random.randint(0, 1000000)
    
    env = B_ACE_TaskEnv( convert_action_space = True,
                                    device = "cpu",
                                    **B_ACE_Config)
    
    #env.action_space = env.action_space()
    #env = PettingZooEnv(env)  
    
    return env 

# Function to update the red agent's behavior in the B_ACE_Config dictionary
def update_red_agent_behavior(config, blue_agents_num, red_agents_num, base_behavior, mission, dShot, lCrank, lBreak):
    config["AgentsConfig"]["blue_agents"]["num_agents"] = blue_agents_num
    config["AgentsConfig"]["red_agents"]["num_agents"] = red_agents_num
    
    config["AgentsConfig"]["red_agents"]["base_behavior"] = base_behavior
    config["AgentsConfig"]["red_agents"]["mission"] = mission
    config["AgentsConfig"]["red_agents"]["beh_config"]["dShot"] = dShot
    config["AgentsConfig"]["red_agents"]["beh_config"]["lCrank"] = lCrank
    config["AgentsConfig"]["red_agents"]["beh_config"]["lBreak"] = lBreak
    

# Function to compute mean and 95% bootstrap confidence interval
def compute_mean_and_ci(data, confidence_level=0.95):
    mean_value = np.mean(data)
    # Perform bootstrap resampling to compute confidence interval
    res = bootstrap((data,), np.mean, confidence_level=confidence_level, n_resamples=10000, method='basic')
    ci_lower, ci_upper = res.confidence_interval
    return mean_value, ci_lower, ci_upper


c:\Programas\b_ace_env\Lib\site-packages\wandb\analytics\sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)


In [2]:
test_env_num = 1       
seed = 1
episodes =  30
render  = False

#Test Conditions
Blue_test_agents = 2
Red_test_agents = 2

Enemies = ["Duck", "FSM_B1","FSM_G3", "FSM_G10"]
Enemies = ["FSM_B1", "FSM_G3"]


#Training Model selection
Blue_training_agents = 2
Red_training_agents = 2

Training_Bases = ["Duck", "FSM_B1","FSM_G3", "FSM_G10"]
Training_Bases = ["FSM_B1", "FSM_G3"]


# "policy_Task_MHA_B_ACE_B_ACE03241024-165541_1185_BestRew" / 1 x 2 FSMG10

for Enemy_type in Enemies:
    for Training_Enemy in Training_Bases:
                        
        if Blue_training_agents == 1 and Red_training_agents == 1:
            
            if Training_Enemy == "Duck":    
                load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241018-082438_1669_BestRew.pth'
                TCode = 11
                
            elif Training_Enemy == "FSM_B1":
                load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241017-210033_1301_BestRew.pth'
                TCode = 21
                
            elif Training_Enemy == "FSM_G3":            
                load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241019-215530_1893_BestRew.pth'
                TCode = 31
                
            elif Training_Enemy == "FSM_G10":            
                load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241022-144023_1329_BestRew.pth'
                TCode = 41
        
        elif Blue_training_agents == 1 and Red_training_agents == 2:
            
            if Training_Enemy == "Duck":    
                #load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241018-082438_1669_BestRew.pth'
                TCode = 12
                
            elif Training_Enemy == "FSM_B1":
                #load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241017-210033_1301_BestRew.pth'
                TCode = 22
                
            elif Training_Enemy == "FSM_G3":            
                #load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241019-215530_1893_BestRew.pth'
                TCode = 32
                
            elif Training_Enemy == "FSM_G10":            
                load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241024-165541_1185_BestRew.pth'
                TCode = 42
        
        elif Blue_training_agents == 2 and Red_training_agents == 2:
            
            if Training_Enemy == "Duck":    
                #load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241018-082438_1669_BestRew.pth'
                TCode = 13
                
            elif Training_Enemy == "FSM_B1":
                load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241118-103140_573_BestRew.pth'
                TCode = 23
                
            elif Training_Enemy == "FSM_G3":            
                load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241112-084823_1689_BestRew.pth'
                TCode = 33
                
            elif Training_Enemy == "FSM_G10":            
                #load_policy_name = f'policy_Task_MHA_B_ACE_B_ACE03241024-165541_1185_BestRew.pth'
                TCode = 43

        
        
        
        # Update B-ACE Enemy Config
        if Enemy_type == "Duck":
            new_enemy_behavior = "duck"   
            new_mission = "striker"
            new_dShot  = [1.04]
            new_lCrank = [1.06]
            new_lBreak = [1.05]
             
            ECode = 1
            
        elif Enemy_type == "FSM_B1":
            new_enemy_behavior = "baseline1"
            new_mission = "striker"
            new_dShot  = [1.04]
            new_lCrank = [1.06]
            new_lBreak = [1.05]
            ECode = 2
            
        elif Enemy_type == "FSM_G3":
            new_enemy_behavior = "baseline1"
            new_mission = "striker"
           # Data for 3 agents
            new_dShot = [0.50, 0.99, 1.04]
            new_lCrank = [0.98, 0.96, 1.14]
            new_lBreak = [1.17, 0.51, 1.05]                      

            ECode = 3
        
        elif Enemy_type == "FSM_G10":
            new_enemy_behavior = "baseline1"
            new_mission = "striker"
            # Data for 10 agents
            new_dShot = [0.50, 0.99, 1.04, 0.50, 0.99, 0.93, 0.57, 0.50, 0.50, 0.50]
            new_lCrank = [0.64, 0.96, 1.14, 0.69, 0.96, 0.69, 1.07, 0.20, 0.98, 0.69]
            new_lBreak = [1.17, 0.51, 1.05, 0.25, 0.84, 0.51, 0.61, 0.37, 1.17, 0.51]            
            ECode = 4


        # Update the red agent's behavior
        update_red_agent_behavior(
            B_ACE_Config,
            Blue_test_agents,
            Red_test_agents, 
            new_enemy_behavior, 
            new_mission, 
            new_dShot, 
            new_lCrank, 
            new_lBreak
        )
        
        policy, optim, agents = _get_agents()        
        test_envs = DummyVectorEnv([_get_env for _ in range(test_env_num)])         
        np.random.seed(seed)
        torch.manual_seed(seed)
                
        
        # Load the saved checkpoint
        for agent in agents:    
            
            if Policy_Config["same_policy"]:
                model_path = os.path.join("Task_MHA_B_ACEDQN", load_policy_name)                            
            else:
                model_path = os.path.join("Task_MHA_B_ACEDQN", load_policy_name) 

            policy.policies[agent].set_eps(0.00)
            policy.policies[agent].load_state_dict(torch.load(model_path))
            policy.policies[agent].eval()
            
        test_collector = Collector(policy, test_envs, exploration_noise=True)

        results = test_collector.collect(n_episode=episodes)#0.02)#, gym_reset_kwargs={'seed' :2})

        print("Mean: ", np.mean(results.returns))
        print("Std:  " , np.std (results.returns))
        print("Max:  " , np.max( results.returns))
        print("Min:  " , np.min(results.returns))

        #Gets Final Stats
        methods = test_envs.get_env_attr("call_results")  # This returns a list of method references
        results = [method() for method in methods]  # Call each method
        df = pd.DataFrame(results)
        
        data = results

        blue_team_data = []
        red_team_data = []
        general_data = []

        for episode in data:
            for entry in episode:
                blue_team_data.append(entry[0])  # First dictionary: Blue team
                red_team_data.append(entry[1])   # Second dictionary: Red team
                general_data.append(entry[2])    # Third dictionary: General sim data

        # Convert to DataFrames
        df_blue = pd.DataFrame(blue_team_data)
        df_blue = df_blue[df_blue.end_cond.notna()]
        
        df_blue['training_beh'] = Training_Enemy
        df_blue['Blue_test_agents_num'] = Blue_test_agents
        df_blue['Blue_training_agents_num'] = Blue_training_agents                

        df_red = pd.DataFrame(red_team_data)
        df_red = df_red[df_red.end_cond.notna()]  
        df_red['eval_enemy'] = Enemy_type      
        df_blue['Red_test_agents_num'] = Red_test_agents
        df_blue['Red_training_agents_num'] = Red_training_agents            

        df_general = pd.DataFrame(general_data)

        # Merge DataFrames for a complete view (optional)
        df_merged = pd.concat([df_general, df_blue.add_prefix('blue_'), df_red.add_prefix('red_')], axis=1)
        df_merged = df_merged[df_merged.blue_end_cond.notna()]

        # Compute mean and confidence intervals for Blue and Red team metrics
        metrics = ['killed', 'missile', 'mission', 'reward']

        final_results = {'eval_enemy' : [], 'training_beh' : [], 'team': [], 'metric': [], 'mean': [], 'ci_lower': [], 'ci_upper': []}

        for metric in metrics:
            for team, df in [('Blue', df_blue), ('Red', df_red)]:
                mean_value, ci_lower, ci_upper = compute_mean_and_ci(df[metric])
                final_results['eval_enemy'].append(Enemy_type)
                final_results['training_beh'].append(Training_Enemy)
                final_results['team'].append(team)
                final_results['metric'].append(metric)
                final_results['mean'].append(mean_value)
                final_results['ci_lower'].append(ci_lower)
                final_results['ci_upper'].append(ci_upper)

        # Convert the results into a DataFrame for display
        df_final_results = pd.DataFrame(final_results)
        print(df_final_results)

        df_merged.to_csv(f'{Blue_test_agents}x{Red_test_agents}_{TCode}.{ECode}.Merged_Eval_{Training_Enemy}_Training_{Enemy_type}_Enemy.csv')
        df_final_results.to_csv(f'{TCode}.{ECode}.Final_Eval_{Training_Enemy}_Training_{Enemy_type}_Enemy.csv')
        
        # Clean up and close the environments
        test_collector.reset()
        test_envs.close()
        

waiting for remote GODOT connection on port 13253
connection established
action space {'input': {'action_type': 'continuous', 'size': 4}}
observation space {'obs': {'size': [41], 'space': 'box'}}
waiting for remote GODOT connection on port 13526
connection established
action space {'input': {'action_type': 'continuous', 'size': 4}}
observation space {'obs': {'size': [41], 'space': 'box'}}


C:\Users\andre\AppData\Local\Temp\ipykernel_142972\665638509.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.policies[agent].load_state_dict(torch.load(model_pa

Mean:  32.47866542078507
Std:   0.32068715124778446
Max:   32.935248997176394
Min:   31.54432512674727
  eval_enemy training_beh  team   metric       mean   ci_lower   ci_upper
0     FSM_B1       FSM_B1  Blue   killed   0.000000   0.000000   0.000000
1     FSM_B1       FSM_B1   Red   killed   2.000000   2.000000   2.000000
2     FSM_B1       FSM_B1  Blue  missile   2.066667   1.966667   2.133333
3     FSM_B1       FSM_B1   Red  missile   0.000000   0.000000   0.000000
4     FSM_B1       FSM_B1  Blue  mission   0.000000   0.000000   0.000000
5     FSM_B1       FSM_B1   Red  mission   0.000000   0.000000   0.000000
6     FSM_B1       FSM_B1  Blue   reward  32.478665  32.370740  32.602621
7     FSM_B1       FSM_B1   Red   reward   0.000000   0.000000   0.000000
close message sent
waiting for remote GODOT connection on port 12287
connection established
action space {'input': {'action_type': 'continuous', 'size': 4}}
observation space {'obs': {'size': [41], 'space': 'box'}}
waiting for remo

C:\Users\andre\AppData\Local\Temp\ipykernel_142972\665638509.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.policies[agent].load_state_dict(torch.load(model_pa

Mean:  31.371146884316897
Std:   0.839263863835147
Max:   32.8830249407978
Min:   29.27956559968197
  eval_enemy training_beh  team   metric       mean   ci_lower   ci_upper
0     FSM_B1       FSM_G3  Blue   killed   0.000000   0.000000   0.000000
1     FSM_B1       FSM_G3   Red   killed   2.000000   2.000000   2.000000
2     FSM_B1       FSM_G3  Blue  missile   3.333333   3.066667   3.600000
3     FSM_B1       FSM_G3   Red  missile   0.000000   0.000000   0.000000
4     FSM_B1       FSM_G3  Blue  mission   0.000000   0.000000   0.000000
5     FSM_B1       FSM_G3   Red  mission   0.000000   0.000000   0.000000
6     FSM_B1       FSM_G3  Blue   reward  31.371147  31.072980  31.677668
7     FSM_B1       FSM_G3   Red   reward   0.000000   0.000000   0.000000
close message sent
waiting for remote GODOT connection on port 13937
connection established
action space {'input': {'action_type': 'continuous', 'size': 4}}
observation space {'obs': {'size': [41], 'space': 'box'}}
waiting for remote 

C:\Users\andre\AppData\Local\Temp\ipykernel_142972\665638509.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.policies[agent].load_state_dict(torch.load(model_pa

Mean:  10.265479814788554
Std:   20.014542893414205
Max:   32.94473585520268
Min:   -26.145978475481847
  eval_enemy training_beh  team   metric       mean   ci_lower   ci_upper
0     FSM_G3       FSM_B1  Blue   killed   0.000000   0.000000   0.000000
1     FSM_G3       FSM_B1   Red   killed   1.823529   1.647059   2.000000
2     FSM_G3       FSM_B1  Blue  missile   3.058824   2.235294   3.705882
3     FSM_G3       FSM_B1   Red  missile   0.117647  -0.058824   0.235294
4     FSM_G3       FSM_B1  Blue  mission   0.000000   0.000000   0.000000
5     FSM_G3       FSM_B1   Red  mission   0.176471   0.000000   0.352941
6     FSM_G3       FSM_B1  Blue   reward  21.051343  11.538524  31.107161
7     FSM_G3       FSM_B1   Red   reward   0.000000   0.000000   0.000000
close message sent
waiting for remote GODOT connection on port 11677
connection established
action space {'input': {'action_type': 'continuous', 'size': 4}}
observation space {'obs': {'size': [41], 'space': 'box'}}
waiting for rem

C:\Users\andre\AppData\Local\Temp\ipykernel_142972\665638509.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.policies[agent].load_state_dict(torch.load(model_pa

Mean:  18.998745893608778
Std:   15.02178144352105
Max:   31.887658285650748
Min:   -6.072678667755855
  eval_enemy training_beh  team   metric       mean   ci_lower   ci_upper
0     FSM_G3       FSM_G3  Blue   killed   0.000000   0.000000   0.000000
1     FSM_G3       FSM_G3   Red   killed   1.909091   1.818182   2.045455
2     FSM_G3       FSM_G3  Blue  missile   4.318182   3.590909   5.000000
3     FSM_G3       FSM_G3   Red  missile   0.090909  -0.045455   0.181818
4     FSM_G3       FSM_G3  Blue  mission   0.000000   0.000000   0.000000
5     FSM_G3       FSM_G3   Red  mission   0.000000   0.000000   0.000000
6     FSM_G3       FSM_G3  Blue   reward  27.659283  25.682322  30.032230
7     FSM_G3       FSM_G3   Red   reward   0.000000   0.000000   0.000000
close message sent
